# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.469969741728446),
 (1, 2.469969741728446),
 (2, 2.469969741728446),
 (3, 2.469969741728446),
 (4, 2.469969741728446)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('what', 10)]),
 (1, [('a', 2), ('is', 18), ('it', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.009325765027667465),
   (None, 0.03446646969359268),
   (None, 0.03792538746967333),
   (None, 0.07342778254068916),
   (None, 0.0968676420982374),
   (None, 0.16151215297839372),
   (None, 0.16674924068003716),
   (None, 0.17352434522054971),
   (None, 0.18127715038885106),
   (None, 0.19975852666773308),
   (None, 0.2138185336168993),
   (None, 0.2555421333925434),
   (None, 0.29762502377751754),
   (None, 0.3081339028662574),
   (None, 0.3229944231369988),
   (None, 0.3773635864959637),
   (None, 0.4087249444125036),
   (None, 0.4598927289973461)]),
 (1,
  [(None, 0.5342198278090078),
   (None, 0.5967188809991392),
   (None, 0.6164230973829381),
   (None, 0.71859885174973),
   (None, 0.7297848079358737),
   (None, 0.8099248379077906),
   (None, 0.8606459942631931),
   (None, 0.9278904831191764),
   (None, 0.9619783296010332),
   (None, 0.9834848649864824),
   (None, 0.9871084912728156),
   (None, 0.998405990674578)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [26]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count, distance=200):
    return [random.randint(0, distance) for i in range(count)]


record = RECORDREADER(count=100)
print('records:\n', record, '\n')

# разбиение на пачки
part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print('records partitional:\n', record_partitional, '\n')

print('max_num:\n')
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

records:
 [61, 170, 30, 65, 5, 58, 148, 88, 65, 94, 179, 198, 59, 181, 1, 71, 157, 0, 174, 4, 187, 66, 189, 81, 191, 118, 87, 166, 74, 139, 52, 154, 49, 53, 183, 199, 144, 184, 82, 153, 160, 76, 21, 144, 113, 112, 152, 118, 50, 166, 186, 121, 154, 152, 61, 196, 155, 192, 134, 102, 76, 62, 64, 169, 15, 68, 16, 2, 135, 12, 91, 111, 136, 101, 73, 6, 49, 109, 83, 14, 75, 83, 182, 28, 171, 90, 158, 173, 169, 103, 23, 188, 185, 51, 36, 0, 197, 97, 54, 182] 

records partitional:
 [[61, 170, 30, 65, 5], [58, 148, 88, 65, 94], [179, 198, 59, 181, 1], [71, 157, 0, 174, 4], [187, 66, 189, 81, 191], [118, 87, 166, 74, 139], [52, 154, 49, 53, 183], [199, 144, 184, 82, 153], [160, 76, 21, 144, 113], [112, 152, 118, 50, 166], [186, 121, 154, 152, 61], [196, 155, 192, 134, 102], [76, 62, 64, 169, 15], [68, 16, 2, 135, 12], [91, 111, 136, 101, 73], [6, 49, 109, 83, 14], [75, 83, 182, 28, 171], [90, 158, 173, 169, 103], [23, 188, 185, 51, 36], [0, 197, 97, 54, 182]] 

max_num:

199


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [28]:
# одинаковый ключ группировки для всех чисел
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count, distance=200):
    return [random.randint(0, distance) for i in range(count)]

records = RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), records))
print('(ключ, число)\n', map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print('группировка:\n', shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print('среднее:\n', reduce_output)

(ключ, число)
 [(1, 139), (1, 54), (1, 92), (1, 76), (1, 75), (1, 86), (1, 190), (1, 196), (1, 59), (1, 183), (1, 37), (1, 34), (1, 59), (1, 62), (1, 200), (1, 149), (1, 130), (1, 199), (1, 85), (1, 32), (1, 4), (1, 191), (1, 157), (1, 199), (1, 23), (1, 169), (1, 127), (1, 144), (1, 64), (1, 40), (1, 72), (1, 2), (1, 192), (1, 99), (1, 90), (1, 21), (1, 119), (1, 145), (1, 151), (1, 127), (1, 88), (1, 113), (1, 141), (1, 84), (1, 171), (1, 73), (1, 139), (1, 80), (1, 190), (1, 50), (1, 70), (1, 148), (1, 92), (1, 74), (1, 189), (1, 147), (1, 110), (1, 149), (1, 18), (1, 84), (1, 40), (1, 77), (1, 151), (1, 169), (1, 151), (1, 96), (1, 181), (1, 31), (1, 111), (1, 145), (1, 32), (1, 101), (1, 68), (1, 168), (1, 190), (1, 50), (1, 69), (1, 9), (1, 27), (1, 65), (1, 181), (1, 183), (1, 74), (1, 35), (1, 106), (1, 33), (1, 55), (1, 18), (1, 182), (1, 73), (1, 119), (1, 186), (1, 95), (1, 9), (1, 143), (1, 77), (1, 0), (1, 185), (1, 40), (1, 194)]
группировка:
 [(1, [139, 54, 92, 76, 75, 8

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [21]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count, distance=200):
    return [random.randint(0, distance) for i in range(count)]

print('Map:\n')
map_output = list(map(lambda x: MAP(x), RECORDREADER(count=100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('группировка:\n')
print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print('среднее:\n')
print(reduce_output)

Map:

[(1, 7), (1, 170), (1, 147), (1, 123), (1, 107), (1, 80), (1, 14), (1, 52), (1, 200), (1, 109), (1, 89), (1, 103), (1, 68), (1, 178), (1, 109), (1, 82), (1, 184), (1, 131), (1, 3), (1, 170), (1, 110), (1, 51), (1, 47), (1, 140), (1, 1), (1, 175), (1, 86), (1, 7), (1, 35), (1, 125), (1, 46), (1, 187), (1, 185), (1, 111), (1, 74), (1, 140), (1, 178), (1, 97), (1, 158), (1, 125), (1, 178), (1, 134), (1, 88), (1, 77), (1, 70), (1, 101), (1, 183), (1, 170), (1, 34), (1, 105), (1, 15), (1, 54), (1, 186), (1, 82), (1, 30), (1, 176), (1, 96), (1, 143), (1, 99), (1, 77), (1, 85), (1, 141), (1, 112), (1, 184), (1, 110), (1, 26), (1, 40), (1, 94), (1, 3), (1, 43), (1, 6), (1, 17), (1, 68), (1, 39), (1, 127), (1, 8), (1, 196), (1, 159), (1, 179), (1, 103), (1, 169), (1, 48), (1, 5), (1, 84), (1, 78), (1, 138), (1, 190), (1, 64), (1, 146), (1, 109), (1, 162), (1, 68), (1, 96), (1, 72), (1, 71), (1, 173), (1, 54), (1, 96), (1, 177), (1, 194)]
группировка:

[(1, [7, 170, 147, 123, 107, 80, 14, 

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [33]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'banana', 'what']), (1, ['a', 'is', 'it'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [36]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    # print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count, distance=100):
    return [(random.randint(0, distance), random.randint(0, distance), random.randint(0, distance)) for i in range(count)]

record = RECORDREADER(100)
print('Кортежи:\n')
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]

print('Кортежи по пачкам:\n')
print(record_partitional)

print('Применение функции индентичности:\n')
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

Кортежи:

[(45, 56, 70), (46, 49, 36), (79, 25, 1), (22, 40, 40), (39, 71, 13), (11, 4, 29), (65, 63, 45), (43, 13, 31), (8, 11, 90), (63, 68, 52), (43, 54, 77), (63, 70, 39), (33, 29, 95), (65, 12, 17), (60, 80, 10), (50, 69, 98), (90, 86, 59), (64, 85, 53), (58, 29, 36), (30, 51, 44), (48, 28, 75), (18, 26, 52), (83, 6, 16), (22, 0, 12), (83, 0, 100), (77, 65, 55), (99, 21, 52), (26, 18, 49), (33, 62, 66), (100, 72, 7), (94, 4, 40), (13, 100, 66), (68, 14, 72), (20, 70, 48), (29, 16, 33), (5, 59, 94), (87, 21, 10), (41, 91, 40), (22, 50, 90), (28, 61, 35), (42, 66, 41), (87, 65, 34), (2, 64, 59), (19, 80, 40), (39, 77, 69), (54, 27, 94), (5, 78, 55), (36, 59, 38), (89, 85, 3), (57, 95, 83), (49, 96, 99), (82, 38, 76), (34, 74, 100), (37, 87, 65), (25, 89, 90), (43, 30, 92), (49, 64, 75), (24, 46, 79), (44, 1, 91), (65, 11, 40), (30, 90, 6), (42, 58, 80), (90, 51, 26), (15, 45, 24), (3, 63, 81), (16, 40, 90), (83, 65, 98), (51, 82, 66), (22, 17, 8), (44, 81, 99), (65, 37, 18), (51, 59

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [39]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count, distance=50):
    return [(random.randint(0, distance), random.randint(0, distance), random.randint(0, distance)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)

print('Кортежи:\n')
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

print('Мап:\n')
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('Группировка:\n')
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print('Проекция:\n')
print(reduce_output)

Кортежи:

[(31, 15, 15), (14, 22, 2), (28, 50, 1), (30, 14, 34), (36, 5, 42), (10, 28, 15), (49, 20, 40), (37, 16, 6), (2, 36, 50), (3, 3, 37), (11, 35, 36), (8, 16, 23), (50, 22, 3), (22, 43, 41), (28, 4, 41), (18, 25, 28), (32, 38, 5), (24, 47, 44), (15, 41, 1), (37, 10, 13), (5, 24, 15), (43, 28, 47), (12, 25, 50), (4, 43, 1), (7, 47, 48), (24, 46, 33), (6, 19, 21), (0, 41, 12), (34, 2, 8), (0, 35, 33), (45, 9, 30), (14, 24, 7), (48, 10, 0), (33, 33, 19), (42, 33, 30), (15, 39, 15), (25, 21, 7), (7, 22, 15), (48, 34, 26), (50, 13, 33), (5, 22, 1), (40, 48, 14), (33, 11, 38), (27, 17, 39), (5, 14, 44), (19, 28, 26), (29, 39, 22), (19, 7, 40), (27, 48, 40), (39, 29, 47), (21, 29, 4), (26, 19, 44), (29, 22, 22), (29, 30, 5), (4, 40, 10), (19, 25, 15), (40, 27, 23), (20, 15, 44), (28, 37, 14), (33, 1, 35), (29, 2, 44), (16, 17, 13), (18, 27, 14), (30, 16, 19), (38, 41, 50), (23, 11, 43), (49, 42, 2), (47, 37, 9), (26, 46, 31), (48, 9, 17), (31, 24, 8), (9, 47, 26), (39, 10, 47), (50, 37

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [40]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print('Кортежи:\n')
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

print('Мап:\n')
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('Группировка:\n')
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print('Объединение:\n')
print(reduce_output)

Кортежи:

[(19, 76, 87), (6, 88, 34), (20, 26, 81), (88, 92, 65), (80, 45, 97), (97, 46, 55), (86, 59, 15), (20, 39, 69), (49, 44, 94), (69, 32, 21), (56, 92, 25), (97, 67, 28), (11, 10, 9), (30, 94, 82), (96, 81, 28), (4, 5, 30), (26, 84, 0), (49, 8, 20), (40, 72, 94), (66, 53, 50), (26, 60, 63), (70, 53, 18), (95, 12, 1), (67, 22, 67), (21, 97, 87), (62, 77, 84), (82, 18, 27), (64, 5, 71), (69, 11, 56), (44, 96, 23), (87, 52, 15), (9, 51, 60), (25, 51, 92), (11, 51, 42), (86, 18, 50), (64, 44, 57), (15, 78, 7), (21, 29, 72), (78, 29, 92), (96, 69, 65), (84, 80, 48), (13, 66, 89), (37, 54, 55), (80, 65, 90), (64, 8, 61), (26, 9, 77), (65, 84, 58), (21, 46, 19), (59, 43, 59), (1, 100, 11), (92, 46, 58), (50, 94, 86), (59, 34, 13), (3, 64, 68), (4, 85, 93), (41, 27, 20), (12, 13, 78), (43, 25, 74), (43, 81, 100), (59, 48, 49), (50, 41, 96), (84, 52, 55), (46, 3, 82), (86, 43, 56), (46, 53, 78), (89, 98, 90), (43, 53, 33), (13, 21, 35), (85, 43, 6), (67, 25, 11), (24, 41, 72), (47, 74, 6

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [42]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print('Кортежи:\n')
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

print('Кортежи:\n')
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('Группировка:\n')
print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print('Пересечение:\n')
print(reduce_output)

Кортежи:

[(2, 0), (2, 3), (2, 0), (2, 1), (1, 1), (1, 2), (0, 2), (3, 1), (3, 3), (1, 0), (0, 1), (3, 2), (0, 0), (3, 3), (0, 2), (3, 3), (0, 0), (0, 3), (3, 2), (1, 3), (1, 1), (0, 3), (0, 1), (2, 0), (2, 2), (3, 2), (2, 0), (2, 3), (1, 1), (2, 0), (0, 0), (0, 2), (3, 2), (3, 3), (2, 2), (0, 0), (3, 2), (3, 1), (2, 1), (0, 0), (1, 3), (3, 3), (1, 0), (1, 3), (2, 0), (3, 1), (1, 2), (3, 2), (3, 0), (1, 3), (2, 2), (1, 1), (2, 3), (2, 2), (1, 1), (0, 3), (3, 2), (1, 0), (3, 0), (2, 2), (1, 3), (2, 3), (1, 0), (3, 2), (0, 1), (1, 0), (1, 0), (3, 2), (3, 1), (2, 0), (3, 0), (3, 2), (2, 3), (0, 1), (0, 2), (2, 2), (3, 0), (1, 1), (1, 3), (3, 2), (3, 2), (0, 1), (0, 2), (0, 2), (3, 2), (0, 3), (0, 2), (3, 0), (1, 1), (3, 2), (1, 2), (3, 2), (1, 3), (3, 0), (3, 3), (0, 1), (3, 2), (0, 1), (2, 3), (1, 1)]
Кортежи:

[((3, 2), (3, 2)), ((1, 1), (1, 1)), ((0, 2), (0, 2)), ((0, 2), (0, 2)), ((3, 2), (3, 2)), ((0, 2), (0, 2)), ((2, 0), (2, 0)), ((2, 1), (2, 1)), ((1, 0), (1, 0)), ((3, 2), (3, 2))

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [44]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)

# print('Кортежи:\n')
# print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

print('Мап:\n')
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('Группировка:\n')
print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]

print('Разница:\n')
print(reduce_output)

Мап:

[(((3, 2), (2, 3), (3, 3)), 2), (((3, 2), (3, 1), (3, 0)), 2), (((2, 2), (0, 1), (0, 0)), 2), (((3, 0), (0, 1), (1, 1)), 2), (((2, 3), (2, 2), (1, 1)), 1), (((1, 0), (0, 0), (0, 1)), 2), (((0, 2), (3, 3), (2, 1)), 2), (((0, 2), (0, 1), (2, 2)), 1), (((0, 2), (0, 0), (2, 1)), 2), (((2, 3), (0, 2), (0, 0)), 1), (((3, 3), (3, 1), (1, 1)), 2), (((2, 0), (3, 3), (3, 0)), 2), (((1, 1), (2, 2), (1, 3)), 2), (((0, 3), (2, 1), (2, 0)), 2), (((1, 2), (1, 0), (1, 2)), 2), (((3, 1), (3, 2), (0, 3)), 2), (((0, 3), (1, 0), (0, 0)), 1), (((1, 1), (2, 2), (3, 3)), 1), (((1, 2), (1, 0), (1, 2)), 2), (((2, 0), (1, 0), (1, 1)), 2), (((1, 2), (1, 2), (2, 3)), 1), (((2, 1), (2, 0), (3, 1)), 2), (((2, 3), (0, 1), (0, 3)), 2), (((0, 0), (3, 1), (0, 1)), 1), (((2, 2), (0, 1), (0, 2)), 2), (((3, 0), (1, 0), (3, 2)), 1), (((3, 3), (3, 0), (0, 1)), 2), (((2, 3), (1, 0), (0, 3)), 1), (((3, 2), (1, 3), (0, 2)), 1), (((2, 2), (0, 3), (2, 1)), 2), (((3, 3), (1, 2), (2, 2)), 1), (((3, 3), (3, 0), (1, 3)), 2), (

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [45]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print('Кортежи:\n')
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

print('Мап:\n')
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print('Группировка:\n')
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print('Соединение:\n')
print(reduce_output)

Кортежи:

[<__main__.Tuple object at 0x00000258D5474D90>, <__main__.Tuple object at 0x00000258D5474ED0>, <__main__.Tuple object at 0x00000258D548C650>, <__main__.Tuple object at 0x00000258D548CA50>, <__main__.Tuple object at 0x00000258D548C510>, <__main__.Tuple object at 0x00000258D548C7D0>, <__main__.Tuple object at 0x00000258D548CA10>, <__main__.Tuple object at 0x00000258D548CB10>, <__main__.Tuple object at 0x00000258D548CB50>, <__main__.Tuple object at 0x00000258D548C4D0>, <__main__.Tuple object at 0x00000258D548CB90>, <__main__.Tuple object at 0x00000258D548CBD0>, <__main__.Tuple object at 0x00000258D548CC50>, <__main__.Tuple object at 0x00000258D548CC90>, <__main__.Tuple object at 0x00000258D548CCD0>, <__main__.Tuple object at 0x00000258D548CD10>, <__main__.Tuple object at 0x00000258D548CD50>, <__main__.Tuple object at 0x00000258D548CD90>, <__main__.Tuple object at 0x00000258D548CDD0>, <__main__.Tuple object at 0x00000258D548CE10>, <__main__.Tuple object at 0x00000258D548CE50>, <_

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [85]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(3, 1, 1), (0, 1, 1), (1, 0, 3), (1, 0, 3), (1, 1, 2), (2, 0, 2), (2, 2, 1), (1, 3, 2), (3, 2, 3), (2, 1, 3), (3, 3, 3), (0, 0, 1), (2, 0, 2), (1, 1, 0), (3, 2, 2), (3, 2, 1), (2, 0, 0), (1, 2, 1), (2, 1, 1), (1, 0, 3), (3, 3, 2), (2, 3, 1), (0, 2, 3), (2, 3, 1), (0, 3, 3), (2, 2, 2), (0, 1, 3), (2, 3, 3), (0, 0, 1), (3, 0, 0), (2, 0, 3), (1, 1, 0), (2, 1, 3), (3, 0, 1), (2, 3, 0), (2, 1, 3), (3, 1, 1), (3, 1, 0), (2, 3, 0), (1, 1, 0), (1, 3, 0), (3, 3, 2), (0, 0, 1), (1, 0, 2), (2, 3, 0), (1, 3, 3), (0, 1, 3), (0, 0, 0), (3, 1, 3), (0, 0, 1), (2, 2, 3), (2, 0, 3), (2, 2, 1), (3, 1, 2), (2, 0, 2), (2, 3, 0), (0, 0, 0), (0, 3, 3), (1, 2, 0), (1, 1, 0), (3, 3, 3), (1, 0, 0), (0, 2, 0), (2, 0, 1), (2, 2, 1), (0, 3, 2), (2, 1, 1), (0, 2, 0), (0, 1, 0), (1, 2, 2), (0, 2, 3), (0, 3, 2), (3, 3, 0), (0, 2, 0), (0, 2, 1), (0, 1, 0), (3, 0, 2), (2, 1, 1), (3, 1, 2), (1, 0, 3), (2, 1, 3), (0, 0, 0), (1, 2, 0), (3, 0, 0), (3, 3, 3), (0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 2), (1, 0, 0), (3, 3, 2)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [86]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [87]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [88]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [89]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [90]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [91]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.004882209977262622), ((0, 1), 0.0038683451450971613), ((0, 2), 0.003390957747262682), ((0, 3), 0.003214427472294149), ((0, 4), 0.0036018560350790443), ((0, 5), 0.0009688484558747747), ((0, 6), 0.004937945352445722), ((0, 7), 0.00024106855149476318), ((0, 8), 0.0036262309305828667), ((0, 9), 0.0028645230944768893), ((0, 10), 0.0029622096829746633), ((0, 11), 0.000612627135925204), ((0, 12), 0.004594738074563853), ((0, 13), 0.0033357819100331572), ((0, 14), 0.0005310222298188597), ((0, 15), 0.0049873557765213045), ((0, 16), 0.0026101958816150484), ((0, 17), 0.0052606402573071335), ((0, 18), 0.003688775428009355), ((0, 19), 0.0035654339981394064), ((0, 20), 0.0011257624299577998), ((0, 21), 0.00515052073444553), ((0, 22), 0.0007713453302701629), ((0, 23), 0.0032375162015068435), ((0, 24), 0.003396008622807865), ((0, 25), 0.0018057050375171655), ((0, 26), 0.004540959918195643), ((0, 27), 0.0026251648417825128), ((0, 28), 0.0049

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [92]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.2597241050229439), ((0, 1), 0.401575155675791), ((0, 2), 0.48696056075347904), ((0, 3), 0.15550329099934523), ((0, 4), 0.6474419040648105), ((0, 5), 0.21157506698264378), ((0, 6), 0.5493977409722797), ((0, 7), 0.12556834078574267), ((0, 8), 0.48166764407265433), ((0, 9), 0.14121495297756564), ((0, 10), 0.21719559880892286), ((0, 11), 0.44477801190453703), ((0, 12), 0.46100539969532117), ((0, 13), 0.6762059090446996), ((0, 14), 0.33092172487529037), ((0, 15), 0.15605497432927276), ((0, 16), 0.0017540311672767492), ((0, 17), 0.07559319004211063), ((0, 18), 0.5329479878291786), ((0, 19), 0.33301178653165975), ((0, 20), 0.38581091716718885), ((0, 21), 0.5607321278530568), ((0, 22), 0.647414618432631), ((0, 23), 0.13884507959910106), ((0, 24), 0.6556002174363336), ((0, 25), 0.6164376851333723), ((0, 26), 0.5012853954486918), ((0, 27), 0.5336417651420756), ((0, 28), 0.6648073794634358), ((0, 29), 0.10175440197720324), ((0, 30), 0

True